# SmartBeds - Preprocesado de Datos

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [1]:
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np

### Lectura

In [11]:
import loadData as ld

datos = ld.load(r"../data/")

datos = ld.preprocess(datos)

### Datos manuales

In [12]:
datos = ld.newSeizure(datos,'2018-11-10 3:30:00',60*20)

### Preprocesado

In [13]:
# Ordención
datos = datos.sort_values(by='DateTime')

In [14]:
# Eliminación de negativos y filtrado
for d in datos:
    if datos[d].dtype == np.float64:
        datos.loc[datos[d]<5.0,d]=0.0

In [20]:
def normalize(tubes):
    """Normalize min-max all float64 columns
    Param dataframe: dataframe with all data
    Return dataframe with normalize data
    """    
    maxs = np.max(tubes)
    mins = np.min(tubes)
    rang = maxs-mins
    
    tubesNorm = (tubes - mins) / rang
    
    return tubesNorm
    

In [25]:
# Normalización
tubesNorm = normalize(datos.iloc[:,1:13])
datos=pd.concat([datos['DateTime'],tubesNorm*100,datos['target']],axis=1)
datos.head()

,DateTime,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,target
4041041,2018-10-30 21:20:23,0.0,27.835052,30.208333,22.772277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041042,2018-10-30 21:20:23,0.0,26.804124,29.166667,21.782178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041043,2018-10-30 21:20:23,0.0,25.773196,28.125000,21.782178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041044,2018-10-30 21:20:24,0.0,26.804124,29.166667,22.772277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041045,2018-10-30 21:20:25,0.0,26.804124,30.208333,22.772277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [20]:
# Suavizado
from scipy.signal import lfilter

#TODO

### Cálculo de las  estadísticas móviles
#### Widgets

In [6]:
import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(
    value=25,
    min=1,
    max=50,
    description='Ventana:',
    readout=True,
    readout_format='d'
)

button = widgets.Button(
    description='Calcular'
)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=24,
    step=1,
    description='Procesado:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

hb = widgets.HBox([slider,button,progress])

In [7]:
display(hb)

dataE = datos.copy()

def estadisticos(b):
    global slider, dataE, progress
    
    progress.value = 0
    
    before = b.description
    b.description='Espera'

    print(" "*100,end='\r')
    
    ventanaMedia = slider.value
    
    for i in range(1,13):
        nombre_col = 'P'+str(i)
        nombre_col_mean = nombre_col+' mean'
        dataE[nombre_col_mean] = dataE[nombre_col].rolling(ventanaMedia).mean()
        progress.value += 1

    ventanaDesviacion = ventanaMedia

    for i in range(1,13):
        nombre_col = 'P'+str(i)
        nombre_col_std = nombre_col+' std'
        dataE[nombre_col_std] = dataE[nombre_col].rolling(ventanaDesviacion).std()
        progress.value += 1
        
    print('Calculos realizados para ventana a',ventanaMedia,end="\r")
    b.description = before
        
    dataE = dataE.dropna()
    

button.on_click(estadisticos)


In [26]:
datos.head()

,DateTime,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,P11,P12,target
4041041,2018-10-30 21:20:23,0.0,27.0,29.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041042,2018-10-30 21:20:23,0.0,26.0,28.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041043,2018-10-30 21:20:23,0.0,25.0,27.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041044,2018-10-30 21:20:24,0.0,26.0,28.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
4041045,2018-10-30 21:20:25,0.0,26.0,29.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


### Exportación

In [28]:
import pickle as pk

In [29]:
with open('datos.pdd','wb') as f:
    pk.dump(datos,f)
    
with open('datosE.pdd','wb') as f:
    pk.dump(dataE,f)